In [20]:
import numpy as np
import pandas as pd
from py_vollib_vectorized import get_all_greeks
import plotly as py
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)

In [21]:
np.linspace(0,2000,100)

array([   0.        ,   20.2020202 ,   40.4040404 ,   60.60606061,
         80.80808081,  101.01010101,  121.21212121,  141.41414141,
        161.61616162,  181.81818182,  202.02020202,  222.22222222,
        242.42424242,  262.62626263,  282.82828283,  303.03030303,
        323.23232323,  343.43434343,  363.63636364,  383.83838384,
        404.04040404,  424.24242424,  444.44444444,  464.64646465,
        484.84848485,  505.05050505,  525.25252525,  545.45454545,
        565.65656566,  585.85858586,  606.06060606,  626.26262626,
        646.46464646,  666.66666667,  686.86868687,  707.07070707,
        727.27272727,  747.47474747,  767.67676768,  787.87878788,
        808.08080808,  828.28282828,  848.48484848,  868.68686869,
        888.88888889,  909.09090909,  929.29292929,  949.49494949,
        969.6969697 ,  989.8989899 , 1010.1010101 , 1030.3030303 ,
       1050.50505051, 1070.70707071, 1090.90909091, 1111.11111111,
       1131.31313131, 1151.51515152, 1171.71717172, 1191.91919

In [22]:
def plot():
    fig = make_subplots(5,1)
    x = np.arange(0,2000,100)

    trace0 = go.Scatter(x = x,
                    y = pnl,
                    mode='lines',
                    name='pnl')
    trace1 = go.Scatter(x = x,
                    y = delta,
                    mode='lines',
                    name='delta')
    trace2 = go.Scatter(x = x,
                    y = gamma,
                    mode='lines',
                    name='gamma')
    trace3 = go.Scatter(x = x,
                    y = theta,
                    mode='lines',
                    name='theta')
    trace4 = go.Scatter(x = x,
                    y = gamma,
                    mode='lines',
                    name='vega')
    fig.add_trace(trace0,1,1)
    fig.add_trace(trace1,2,1)
    fig.add_trace(trace2,3,1)
    fig.add_trace(trace3,4,1)
    fig.add_trace(trace4,5,1)

    py.offline.iplot(fig)

In [23]:
# 牛市看涨
# call：平值多头 + 虚值空头

s = 1000
k1,k2 = 1000,1500
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(0,2000,100):
    pnl1 = (i - k1 ) if i > k1 else 0
    pnl2 = 0 if i<k2 else (k2-i)
    pl = pnl1 + pnl2
    pnl.append(pl)

# delta
delta=[]
for i in range(0,2000,100):
    delta.append(get_all_greeks('c',i,k1,t,0,sigma)['delta'][0]-get_all_greeks('c',i,k2 ,t,0,sigma)['delta'][0])

# Gamma, Theta, Vega
gamma,theta,vega = [],[],[]
for i in range(0,2000,100):
    gamma.append(get_all_greeks('c',i,k1,t,0,sigma)['gamma'][0]-get_all_greeks('c',i,k2 ,t,0,sigma)['gamma'][0])
    theta.append(get_all_greeks('c',i,k1,t,0,sigma)['theta'][0]-get_all_greeks('c',i,k2 ,t,0,sigma)['theta'][0])
    vega.append(get_all_greeks('c',i,k1,t,0,sigma)['vega'][0]-get_all_greeks('c',i,k2 ,t,0,sigma)['vega'][0])

plot()

In [24]:
# 熊市看跌
# put：平值多头 + 虚值空头

s = 1000
k1,k2 = 1000,500
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(0,2000,100):
    pnl1 = (k1 -i) if i < k1 else 0
    pnl2 = 0 if i > k2 else (i-k2)
    pl = pnl1 + pnl2
    pnl.append(pl)

# delta
delta=[]
for i in range(0,2000,100):
    delta.append(get_all_greeks('p',i,k1,t,0,sigma)['delta'][0]-get_all_greeks('p',i,k2 ,t,0,sigma)['delta'][0])

# Gamma, Theta, Vega
gamma,theta,vega = [],[],[]
for i in range(0,2000,100):
    gamma.append(get_all_greeks('p',i,k1,t,0,sigma)['gamma'][0]-get_all_greeks('p',i,k2 ,t,0,sigma)['gamma'][0])
    theta.append(get_all_greeks('p',i,k1,t,0,sigma)['theta'][0]-get_all_greeks('p',i,k2 ,t,0,sigma)['theta'][0])
    vega.append(get_all_greeks('p',i,k1,t,0,sigma)['vega'][0]-get_all_greeks('p',i,k2 ,t,0,sigma)['vega'][0])

plot()

In [25]:
# 蝶式多头
# long ITM call + short OTM call X2 + long OTM call

s = 1000
k1,k2,k3 = 800,1000,1200
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(0,2000,100):
    pnl1 = (i - k1) if i > k1 else 0
    pnl2 = 0 if i < k2 else (k2-i) * 2
    pnl3 = (i - k3) if i > k3 else 0
    pl = pnl1 + pnl2 + pnl3
    pnl.append(pl)

# delta
delta=[]
for i in range(0,2000,100):
    delta.append(get_all_greeks('c',i,k1,t,0,sigma)['delta'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['delta'][0] * 2 + get_all_greeks('c',i,k3,t,0,sigma)['delta'][0])

# Gamma, Theta, Vega
gamma,theta,vega = [],[],[]
for i in range(0,2000,100):
    gamma.append(get_all_greeks('c',i,k1,t,0,sigma)['gamma'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['gamma'][0] * 2 + get_all_greeks('c',i,k3,t,0,sigma)['gamma'][0])
    theta.append(get_all_greeks('c',i,k1,t,0,sigma)['theta'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['theta'][0] * 2 + get_all_greeks('c',i,k3,t,0,sigma)['theta'][0])
    vega.append(get_all_greeks('c',i,k1,t,0,sigma)['vega'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['vega'][0] * 2 + get_all_greeks('c',i,k3,t,0,sigma)['vega'][0])

plot()

In [26]:
# 蝶式空头
# short ITM call + long ATM call X2 + short OTM call

s = 1000
k1,k2,k3 = 800,1000,1200
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(0,2000,100):
    pnl1 = (k1 - i) if i > k1 else 0
    pnl2 = (i - k2) * 2 if i > k2 else 0
    pnl3 = (k3 - i) if i > k3 else 0
    pl = pnl1 + pnl2 + pnl3
    pnl.append(pl)

# delta
delta=[]
for i in range(0,2000,100):
    delta.append(-get_all_greeks('c',i,k1,t,0,sigma)['delta'][0] + get_all_greeks('c',i,k2 ,t,0,sigma)['delta'][0] * 2 - get_all_greeks('c',i,k3,t,0,sigma)['delta'][0])

# Gamma, Theta, Vega
gamma,theta,vega = [],[],[]
for i in range(0,2000,100):
    gamma.append(-get_all_greeks('c',i,k1,t,0,sigma)['gamma'][0] + get_all_greeks('c',i,k2 ,t,0,sigma)['gamma'][0] * 2 - get_all_greeks('c',i,k3,t,0,sigma)['gamma'][0])
    theta.append(-get_all_greeks('c',i,k1,t,0,sigma)['theta'][0] + get_all_greeks('c',i,k2 ,t,0,sigma)['theta'][0] * 2 - get_all_greeks('c',i,k3,t,0,sigma)['theta'][0])
    vega.append(-get_all_greeks('c',i,k1,t,0,sigma)['vega'][0] + get_all_greeks('c',i,k2 ,t,0,sigma)['vega'][0] * 2 - get_all_greeks('c',i,k3,t,0,sigma)['vega'][0])

plot()